# Preprocessing

In [1]:
# Basics
import numpy as np
import pandas as pd
import glob
# To check vessel names
import re
# To get exact lat/long
from shapely.wkt import loads
# To map the ships or coastlines
import geopandas as gpd
from geopy.distance import geodesic
# To find coastline distance
from scipy.spatial import KDTree
# For XgBoosting
import xgboost as xgb
import random
# For upsampling during the modeling
from imblearn.over_sampling import SMOTE

In [9]:
# Data formatting 
this_dir = "/sfs/gpfs/tardis/project/SDS/capstones/ds6013/iuu_fishing/retry2/east_asia"
files = glob.glob(this_dir + '/*.csv')
files.sort()
files = files[1000:1100]

In [3]:
# Set dtypes to avoid errors later
dtypes = {'FID': 'object', 'mmsi': 'int64', 'source_id': 'int64', 'imo': 'float64', 
          'vessel_name': 'object', 'callsign': 'object', 'vessel_type': 'object', 
          'vessel_type_code': 'float64', 'vessel_type_cargo': 'object', 'vessel_class': 'object',
          'length': 'float64', 'width': 'float64', 'flag_country': 'object', 'flag_code': 'float64',
          'destination': 'object', 'eta': 'int64', 'draught': 'float64', 'position': 'object',
          'latitude': 'float64', 'longitude': 'float64', 'sog': 'float64', 'cog': 'float64',
          'rot': 'float64', 'heading': 'int64', 'nav_status': 'object', 'nav_status_code': 'int64',
          'source': 'object', 'ts_pos_utc': 'int64', 'ts_static_utc': 'int64', 
          'dt_pos_utc': 'object', 'dt_static_utc': 'object', 'vessel_type_main': 'float64',
          'vessel_type_sub': 'float64', 'message_type': 'int64', 'eeid': 'float64', 'dtg': 'object'}

# Read each CSV file into a list of DataFrames
dfs = [pd.concat(pd.read_csv(file, dtype = dtypes, chunksize = 100000)) for file in files]

In [4]:
# Concatenate the DataFrames into a single DataFrame
ais = pd.concat(dfs, ignore_index=True).reset_index(drop = True)
ais['dtg'] = pd.to_datetime(ais['dtg'])
ais = ais.query("25 < latitude < 35 & 120 < longitude < 130")
ais = ais.sort_values(['mmsi', 'dtg'])

In [5]:
# Fix NA names
ais.loc[ais['vessel_name'].isnull(), 'vessel_name'] = " "

# Calculate the time difference between consecutive rows for each mmsi
ais['time_diff'] = ais.groupby('mmsi')['dtg'].diff()

# Create a new column called first_occurrence based on the identified conditions
ais['first_occurrence'] = ((ais['time_diff'].isnull()) | (ais['time_diff'] > pd.Timedelta(hours=4))).astype(int)
ais['occ_num'] = ais.groupby('mmsi').agg(occ_num = ('first_occurrence', 'cumsum')) # Separate different trips

# Get distance traveled
ais['old_lat'] = ais.groupby(['mmsi', 'occ_num'])['latitude'].shift(1)
ais['old_long'] = ais.groupby(['mmsi', 'occ_num'])['longitude'].shift(1)

# Get distance and spped from consecutive readings
def calculate_distance(row):
    if pd.notna(row['old_lat']) and pd.notna(row['old_long']) and pd.notna(row['latitude']) and pd.notna(row['longitude']):
        return geodesic((row['old_lat'], row['old_long']), (row['latitude'], row['longitude'])).nautical
    else:
        return pd.NA

ais['distance'] = ais.apply(calculate_distance, axis = 1)
ais['distance'] = pd.to_numeric(ais['distance'], errors='coerce').fillna(0)
ais['speed'] = ais['distance'] / (ais['time_diff'].dt.seconds.replace(0, pd.NA) / 3600)
ais['distance'] = ais.groupby(['mmsi', 'occ_num']).agg(distance = ('distance', 'cumsum'))

In [6]:
### Check naming conventions
# Look for sketchy names
def check_net_name(name):
    name = name.lower()
    net_name = ('%' in name or 
                'buoy' in name or 
                'net' in name or 
                bool(re.search(r"\d+v", name)))
    if net_name is np.nan:
        net_name = 0
    return net_name

ship_names = ais['vessel_name'].astype('str').unique()
net_names = [check_net_name(name) for name in ship_names]
ais['net_name'] = ais['vessel_name']\
    .map(dict(zip(ship_names, net_names)))
ais['net_name'] = ais.groupby(['mmsi', 'occ_num'])['net_name'].transform('max')
# Look for bad mmsi values
ais['mmsi_length'] = ais['mmsi'].astype('str').str.len() != 9
del ship_names, net_names

In [7]:
# Check for starting in water
# Read in coastline data from https://www.naturalearthdata.com/downloads/10m-physical-vectors/
coastline = pd.concat([gpd.read_file("/sfs/gpfs/tardis/project/SDS/capstones/ds6013/iuu_fishing/data/ne_10m_land/ne_10m_land.shp"), 
                       gpd.read_file("/sfs/gpfs/tardis/project/SDS/capstones/ds6013/iuu_fishing/data/ne_10m_minor_islands/ne_10m_minor_islands.shp")])
coastline = coastline.cx[min(ais['longitude'])-1:max(ais['longitude'])+1, 
                         min(ais['latitude'])-1:max(ais['latitude'])+1].geometry
# Make function to find coastline distance

def distance_to_coast(df, coastline):
    # Get coordinates of all of the ships
    ship_coords = list(df.apply(lambda x: (x.longitude, x.latitude), 
                                axis = 1))
    # Get coordinates of the coastlines
    coast_coords = [] 
    for geom in coastline.geometry:
        if geom.geom_type == 'Polygon':
            coast_coords.extend(list(geom.exterior.coords))
        elif geom.geom_type == 'MultiPolygon':
            for polygon in geom.geoms:
                coast_coords.extend(list(polygon.exterior.coords))
    # Use KDTree to find the nearest points to each ship
    tree = KDTree(coast_coords)
    idx = tree.query(ship_coords, k=1)[1]
    coast_points = pd.Series(coast_coords).iloc[idx]
    # Get the distances in nautical miles
    naut_dist = [geodesic((ship[1], ship[0]), (coast[1], coast[0])).nautical 
                 for ship, coast 
                 in zip(ship_coords, list(coast_points))]
    # Insert into the dataframe
    df['distance_to_coast'] = naut_dist

    return df

# Find distance to shore at start
first_time = ais['dtg'].min()
new_vessels = ais.loc[(ais['first_occurrence'] == 1) & (ais['dtg'] - first_time >= pd.Timedelta(hours = 4)), 
                      ['mmsi', 'occ_num', 'latitude', 'longitude']]
new_vessels = distance_to_coast(new_vessels, coastline)
new_vessels['spawn_offshore'] = new_vessels['distance_to_coast'] >= 1

# Make dictionary and map to dataframe
spawn_dict = {}
for index, row in new_vessels.iterrows():
    # Combine mmsi and occ_num to create the key
    key = (row['mmsi'], row['occ_num'])
    # Assign the value of 'spawn_offshore' to the key in the dictionary
    spawn_dict[key] = row['spawn_offshore']
ais['spawn_offshore'] = ais.apply(lambda row: (row['mmsi'], row['occ_num']), axis=1)\
    .map(spawn_dict)
ais['spawn_offshore'] = ais['spawn_offshore'].fillna(False)
del index, row, key, coastline, spawn_dict, new_vessels, first_time

In [8]:
# If it is clearly spoofing location 
ais['spoof'] = ais['speed'] >= 150 # World record is 58.1, so allow for some measurement error 
ais['spoof'] = ais.groupby(['mmsi', 'occ_num'])['spoof'].transform('max')

In [9]:
# Add all potential issues together
ais['red_flags'] = ais['net_name'].astype('int') + ais['mmsi_length'].astype('int') + \
    ais['spawn_offshore'].astype('int') + ais['spoof'].astype('int')
# Transform latitude and longitude to normalized values
min_lat, max_lat = ais['latitude'].min(), ais['latitude'].max()
min_lon, max_lon = ais['longitude'].min(), ais['longitude'].max()


# Normalize latitude and longitude values to the range [0, 1]
ais['x'] = (ais['longitude'] - min_lon) / (max_lon - min_lon)
ais['y'] = (ais['latitude'] - min_lat) / (max_lat - min_lat)
# Get rid of extraneous columns
ais = ais.loc[:, ['mmsi', 'occ_num', 'longitude', 'latitude', 'x', 'y', 'sog', 'cog', 
                  'rot', 'distance', 'dtg', 'net_name', 'mmsi_length', 'spawn_offshore', 
                  'spoof', 'red_flags']]
del min_lat, max_lat, min_lon, max_lon

In [10]:
# Prep data for XGBoost
ais_model_data = ais.groupby(['mmsi', 'occ_num'])\
    .agg(
        net_name = ('net_name', 'max'),
        mmsi_length = ('mmsi_length', 'max'),
        spawn_offshore = ('spawn_offshore', 'max'),
        spoof = ('spoof', 'max'),
        speed_0 = ('sog', 'min'),
        speed_med = ('sog', 'median'),
        speed_99 = ('sog', 'max'),
        speed_std = ('sog', 'std'),
        dist_med = ('distance', 'median'),
        dist_99 = ('distance', 'max'),
        dist_std = ('distance', 'std'), 
        x_0 = ('x', 'min'),
        x_med = ('x', 'median'),
        x_99 = ('x', 'max'),
        x_std = ('x', 'std'),
        y_0 = ('y', 'min'),
        y_med = ('y', 'median'),
        y_99 = ('y', 'max'),
        y_std = ('y', 'std'),
        red_flags = ('red_flags', 'max'),
        entries = ('net_name', 'size')
)
ais_model_data['dist_med'] = pd.to_numeric(ais_model_data['dist_med'], errors='coerce')
ais_model_data['dist_99'] = pd.to_numeric(ais_model_data['dist_99'], errors='coerce')
ais_model_data['dist_std'] = pd.to_numeric(ais_model_data['dist_std'], errors='coerce')
ais_model_data = ais_model_data.fillna(pd.NA) # Set to pd.NA for future reasons
ais_model_data = ais_model_data.reset_index()
#add line for write csv -- save xgboost data
ais_model_data.to_csv('xgboost_preprocessed_data/xgboost_data_newtest2.csv', index=False)